In [1]:
import ciftify
from ciftify.utils import run
import os
import datetime
import logging
import glob

In [3]:
working_dir = '/home/edickie/Documents/ciftify_tests/'
#working_dir = '/scratch/edickie/ciftify_intergration_tests/'
src_data_dir= os.path.join(working_dir,'src_data')
#work_from = '/home/edickie/Documents/ciftify_tests/run_2017-09-02/'
#work_from = '/home/edickie/Documents/ciftify_tests/run_2017-09-02/'
work_from = None

if work_from:
    new_outputs = work_from
else:
    new_outputs= os.path.join(working_dir,'run_{}'.format(datetime.date.today()))
DEBUG = True
DRYRUN = True

## getting the data

freesurfer_webtgz = 'https://s3.amazonaws.com/openneuro/ds000030/ds000030_R1.0.4/compressed/ds000030_R1.0.4_derivatives_freesurfer_sub50004-50008.zip'
func_webtgz = 'https://s3.amazonaws.com/openneuro/ds000030/ds000030_R1.0.4/compressed/ds000030_R1.0.4_derivatives_sub50004-50008.zip'

subids = ['sub-50005','sub-50007']
#subids = ['sub-50005','sub-50006']

logger = logging.getLogger('ciftify')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)  

fh = logging.FileHandler(os.path.join(working_dir, 'ciftify_tests.log'))
fh.setLevel(logging.INFO)
fh.setFormatter(formatter)
logger.addHandler(fh)

fs_subjects_dir = os.path.join(src_data_dir, 'ds000030_R1.0.4',
                               'derivatives','freesurfer')
hcp_data_dir = os.path.join(new_outputs, 'hcp')

if not os.path.exists(hcp_data_dir):
    run(['mkdir','-p',hcp_data_dir])

Running: mkdir -p /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp
Running: mkdir -p /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp


In [4]:
def download_file(web_address, local_filename):
    '''download file if it does not exist'''
    if not os.path.isfile(local_filename):
        run(['wget', web_address, '-O', local_filename])
    if not os.path.getsize(local_filename) > 0:
        os.remove(local_filename)

In [5]:
def get_recon_all_outputs(hcp_data_dir, subid):
    recon_all_out = folder_contents_list(os.path.join(hcp_data_dir, subid))
    recon_all_out = [x.replace(subid, 'subid') for x in recon_all_out]
    return(recon_all_out)

In [6]:
def folder_contents_list(path):
    '''returns a list of folder contents'''
    folder_contents = glob.glob(os.path.join(path, '**'), recursive = True)
    folder_contents = [x.replace('{}/'.format(path),'') for x in folder_contents ]
    folder_contents = folder_contents[1:] ## the first element is the path name
    return(folder_contents)

In [86]:
def seed_corr_default_out(func, seed):
    functype, funcbase = ciftify.io.determine_filetype(func)
    _, seedbase = ciftify.io.determine_filetype(seed)
    outputdir = os.path.dirname(func)
    outbase = '{}_{}'.format(funcbase, seedbase)
    outputname = os.path.join(outputdir, outbase)
    if functype == "nifti": outputname = '{}.nii.gz'.format(outputname)
    if functype == "cifti": outputname = '{}.dscalar.nii'.format(outputname)
    return(outputname)

def run_vis_map(result_map, result_prefix, result_type):
    run(['cifti_vis_map', '{}-snaps'.format(result_type), '--hcp-data-dir', hcp_data_dir, 
     result_map, subid, result_prefix])

def run_seedcorr_peaktable(result_map):
    run(['ciftify_peaktable',  
         '--min-threshold', '-0.5', 
         '--max-threshold', '0.5', 
         '--no-cluster-dlabel', 
         result_map])

def run_seed_corr_fmri_test(func_cifti, hcp_data_dir):
    ''' runs one of the seed corrs and then peak table to get a csv to test for each dtseries'''
    subid = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(func_cifti)))))
    atlas_vol =  os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'wmparc.nii.gz')
    struct = 'RIGHT-PUTAMEN'
    putamen_vol_seed_mask = os.path.join(seed_corr_dir, '{}_{}_vol.nii.gz'.format(subid, struct))
    if not os.path.exists(putamen_vol_seed_mask):
        run(['wb_command', '-volume-label-to-roi', atlas_vol, putamen_vol_seed_mask, '-name', struct])
    run(['ciftify_seed_corr', func_cifti, putamen_vol_seed_mask])
    run_seedcorr_peaktable(seed_corr_default_out(func_cifti, putamen_vol_seed_mask))

# Getting ABIDE and running recon-all

In [7]:
abide_amazon_addy = 'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/freesurfer/5.1'

subid = 'NYU_0050954'
abide_freesurfer = os.path.join(src_data_dir, 'abide', 'freesurfer')

fs_subdir = os.path.join(abide_freesurfer, subid)
if not os.path.exists(fs_subdir):    
    run(['mkdir', '-p', fs_subdir])

for subdir in ['mri', 'surf','label','scripts']:
    localdir = os.path.join(fs_subdir, subdir)
    if not os.path.exists(localdir):
        run(['mkdir', '-p', localdir])
    
for filename in ['T1.mgz', 'aparc+aseg.mgz', 'aparc.a2009s+aseg.mgz', 'wmparc.mgz', 'brain.finalsurfs.mgz']:
    download_file(os.path.join(abide_amazon_addy, subid, 'mri', filename.replace('+','%20')),
                               os.path.join(fs_subdir, 'mri', filename))

for surface in ['pial', 'white', 'sphere.reg', 'sphere', 'curv', 'sulc', 'thickness']:
    for hemi in ['lh', 'rh']:
        download_file(os.path.join(abide_amazon_addy, subid, 'surf', '{}.{}'.format(hemi,surface)),
                      os.path.join(fs_subdir, 'surf', '{}.{}'.format(hemi,surface)))

for labelname in ['aparc', 'aparc.a2009s', 'BA']:
    for hemi in ['lh', 'rh']:
        download_file(os.path.join(abide_amazon_addy, subid, 'label', '{}.{}.annot'.format(hemi,labelname)),
                      os.path.join(fs_subdir, 'label', '{}.{}.annot'.format(hemi,labelname))) 

for script in ['recon-all.done', 'build-stamp.txt']:
    download_file(os.path.join(abide_amazon_addy, subid, 'scripts', script), 
                  os.path.join(fs_subdir, 'scripts', script))

Running: wget https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/freesurfer/5.1/NYU_0050954/mri/aparc%20aseg.mgz -O /home/edickie/Documents/ciftify_tests/src_data/abide/freesurfer/NYU_0050954/mri/aparc+aseg.mgz
Running: wget https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/freesurfer/5.1/NYU_0050954/mri/aparc%20aseg.mgz -O /home/edickie/Documents/ciftify_tests/src_data/abide/freesurfer/NYU_0050954/mri/aparc+aseg.mgz
cmd: wget https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/freesurfer/5.1/NYU_0050954/mri/aparc%20aseg.mgz -O /home/edickie/Documents/ciftify_tests/src_data/abide/freesurfer/NYU_0050954/mri/aparc+aseg.mgz 
 Failed with returncode 8
cmd: wget https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/freesurfer/5.1/NYU_0050954/mri/aparc%20aseg.mgz -O /home/edickie/Documents/ciftify_tests/src_data/abide/freesurfer/NYU_0050954/mri/aparc+aseg.mgz 
 Failed with returncode 8
--2017-09-08 21:27:14-

In [8]:
run(['ciftify_recon_all', '--hcp-data-dir', hcp_data_dir,
        '--fs-subjects-dir', abide_freesurfer, 
        'NYU_0050954'])

Running: ciftify_recon_all --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --fs-subjects-dir /home/edickie/Documents/ciftify_tests/src_data/abide/freesurfer NYU_0050954
Running: ciftify_recon_all --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --fs-subjects-dir /home/edickie/Documents/ciftify_tests/src_data/abide/freesurfer NYU_0050954
/home/edickie/Documents/ciftify_tests/src_data/abide/freesurfer/NYU_0050954/mri/aparc.a2009s+aseg.mgz not found
/home/edickie/Documents/ciftify_tests/src_data/abide/freesurfer/NYU_0050954/mri/aparc+aseg.mgz not found

/home/edickie/Documents/ciftify_tests/src_data/abide/freesurfer/NYU_0050954/mri/aparc.a2009s+aseg.mgz not found
/home/edickie/Documents/ciftify_tests/src_data/abide/freesurfer/NYU_0050954/mri/aparc+aseg.mgz not found



0

In [9]:

run(['cifti_vis_recon_all', 'snaps', '--hcp-data-dir',hcp_data_dir, 'NYU_0050954'])
run(['cifti_vis_recon_all', 'index', '--hcp-data-dir', hcp_data_dir])

Running: cifti_vis_recon_all snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp NYU_0050954
Running: cifti_vis_recon_all snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp NYU_0050954
Running: cifti_vis_recon_all index --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp
Running: cifti_vis_recon_all index --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp


0

## Download ABIDE PCP data for ciftify_vol_result tests

In [10]:
def download_vmhc(subid):
    amazon_addy = 'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/cpac/filt_global/vmhc/{}_vmhc.nii.gz'.format(subid)
    sub_vmhc = os.path.join(src_vmhc, '{}_vmhc.nii.gz'.format(subid))
    download_file(amazon_addy, sub_vmhc)        

src_vmhc = os.path.join(src_data_dir, 'abide', 'vmhc')    
if not os.path.exists(src_vmhc): 
    run(['mkdir', src_vmhc])

subjects=['NYU_0050954','NYU_0050955']
for subid in subjects:
    download_vmhc(subid)

In [ ]:
result_map = os.path.join(seed_corr_dir, '{}_{}_ciftitocifti_masked.dscalar.nii'.format(subid, struct))
result_type = 'cifti'
result_prefix = '{}_{}_ciftitocifti_masked'.format(subid, struct)

In [11]:
subject = 'NYU_0050954'
run(['cifti_vis_map', 'nifti-snaps',
        '--hcp-data-dir', hcp_data_dir,
        '--qcdir', os.path.join(hcp_data_dir, 'abide_vmhc_vis'),
        '--resample-nifti', '--colour-palette', 'fidl', 
        os.path.join(src_vmhc, '{}_vmhc.nii.gz'.format(subject)), subject, '{}_vmhc'.format(subject)])

Running: cifti_vis_map nifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --qcdir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/abide_vmhc_vis --resample-nifti --colour-palette fidl /home/edickie/Documents/ciftify_tests/src_data/abide/vmhc/NYU_0050954_vmhc.nii.gz NYU_0050954 NYU_0050954_vmhc
Running: cifti_vis_map nifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --qcdir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/abide_vmhc_vis --resample-nifti --colour-palette fidl /home/edickie/Documents/ciftify_tests/src_data/abide/vmhc/NYU_0050954_vmhc.nii.gz NYU_0050954 NYU_0050954_vmhc


0

In [12]:
subject = 'NYU_0050954'
run(['cifti_vis_map', 'nifti-snaps',
        '--hcp-data-dir', hcp_data_dir,
        '--qcdir', os.path.join(hcp_data_dir, 'abide_vmhc_vis'),
        '--resample-nifti',  
        os.path.join(src_vmhc, '{}_vmhc.nii.gz'.format(subject)), subject, '{}_vmhc_dcol'.format(subject)])

Running: cifti_vis_map nifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --qcdir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/abide_vmhc_vis --resample-nifti /home/edickie/Documents/ciftify_tests/src_data/abide/vmhc/NYU_0050954_vmhc.nii.gz NYU_0050954 NYU_0050954_vmhc_dcol
Running: cifti_vis_map nifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --qcdir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/abide_vmhc_vis --resample-nifti /home/edickie/Documents/ciftify_tests/src_data/abide/vmhc/NYU_0050954_vmhc.nii.gz NYU_0050954 NYU_0050954_vmhc_dcol


0

In [13]:
subject = 'NYU_0050954'
run(['cifti_vis_map', 'nifti-snaps', '--debug',
        '--qcdir', os.path.join(hcp_data_dir, 'abide_vmhc_vis'),
        '--resample-nifti', '--colour-palette', 'fidl', 
        os.path.join(src_vmhc, '{}_vmhc.nii.gz'.format(subject)), 'HCP_S1200_GroupAvg', '{}_vmhc'.format(subject)])

Running: cifti_vis_map nifti-snaps --debug --qcdir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/abide_vmhc_vis --resample-nifti --colour-palette fidl /home/edickie/Documents/ciftify_tests/src_data/abide/vmhc/NYU_0050954_vmhc.nii.gz HCP_S1200_GroupAvg NYU_0050954_vmhc
Running: cifti_vis_map nifti-snaps --debug --qcdir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/abide_vmhc_vis --resample-nifti --colour-palette fidl /home/edickie/Documents/ciftify_tests/src_data/abide/vmhc/NYU_0050954_vmhc.nii.gz HCP_S1200_GroupAvg NYU_0050954_vmhc
[cifti_vis_map] DEBUG: {'--colour-palette': 'fidl',
 '--debug': True,
 '--hcp-data-dir': None,
 '--help': False,
 '--qcdir': '/home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/abide_vmhc_vis',
 '--resample-nifti': True,
 '--subjects-filter': None,
 '--verbose': False,
 '<map-name>': 'NYU_0050954_vmhc',
 '<map.dscalar.nii>': None,
 '<map.nii>': '/home/edickie/Documents/ciftify_tests/src_data/abide/vmhc/NYU_0050954_vmhc.nii.gz',
 '

[cifti_vis_map] DEBUG: {'--colour-palette': 'fidl',
 '--debug': True,
 '--hcp-data-dir': None,
 '--help': False,
 '--qcdir': '/home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/abide_vmhc_vis',
 '--resample-nifti': True,
 '--subjects-filter': None,
 '--verbose': False,
 '<map-name>': 'NYU_0050954_vmhc',
 '<map.dscalar.nii>': None,
 '<map.nii>': '/home/edickie/Documents/ciftify_tests/src_data/abide/vmhc/NYU_0050954_vmhc.nii.gz',
 '<subject>': 'HCP_S1200_GroupAvg',
 'cifti-snaps': False,
 'index': False,
 'nifti-snaps': True}
[cifti_vis_map] DEBUG: Creating temp dir:/tmp/tmponq7e12t on host:edickie-UX305FA
[ciftify.utils] INFO: Running: ciftify_vol_result --debug --resample-nifti HCP_S1200_GroupAvg /home/edickie/Documents/ciftify_tests/src_data/abide/vmhc/NYU_0050954_vmhc.nii.gz /tmp/tmponq7e12t/NYU_0050954_vmhc.dscalar.nii
[ciftify.utils] INFO: [ciftify_vol_result] INFO: {'--HCP-MSMAll': False,
 '--HCP-Pipelines': False,
 '--debug': True,
 '--dilate': None,
 '--dry-run': False,
 '

0

In [14]:
subject = 'NYU_0050955'
run(['cifti_vis_map', 'nifti-snaps',
        '--qcdir', os.path.join(hcp_data_dir, 'abide_vmhc_vis'),
        '--resample-nifti',
        os.path.join(src_vmhc, '{}_vmhc.nii.gz'.format(subject)), 'HCP_S1200_GroupAvg', '{}_vmhc'.format(subject)])

Running: cifti_vis_map nifti-snaps --qcdir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/abide_vmhc_vis --resample-nifti /home/edickie/Documents/ciftify_tests/src_data/abide/vmhc/NYU_0050955_vmhc.nii.gz HCP_S1200_GroupAvg NYU_0050955_vmhc
Running: cifti_vis_map nifti-snaps --qcdir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/abide_vmhc_vis --resample-nifti /home/edickie/Documents/ciftify_tests/src_data/abide/vmhc/NYU_0050955_vmhc.nii.gz HCP_S1200_GroupAvg NYU_0050955_vmhc


0

In [15]:
run(['cifti_vis_map', 'index',
        '--hcp-data-dir', '/tmp',
        '--qcdir', os.path.join(hcp_data_dir, 'abide_vmhc_vis')])

Running: cifti_vis_map index --hcp-data-dir /tmp --qcdir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/abide_vmhc_vis
Running: cifti_vis_map index --hcp-data-dir /tmp --qcdir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/abide_vmhc_vis


0

# Download the main dataset

In [16]:
subids = ['sub-50005','sub-50007']

if not os.path.exists(src_data_dir):
    run(['mkdir','-p',src_data_dir])

for subid in subids:
    sub_fs_path = os.path.join('ds000030_R1.0.4','derivatives','freesurfer', subid)
if not os.path.exists(os.path.join(src_data_dir, sub_fs_path)):
    if not os.path.exists(os.path.join(src_data_dir, 
                                       os.path.basename(freesurfer_webtgz))):        
        run(['wget', '-P', src_data_dir, freesurfer_webtgz])

    run(['unzip', 
         os.path.join(src_data_dir, 
                      os.path.basename(freesurfer_webtgz)), 
         os.path.join(sub_fs_path,'*'),
         '-d', src_data_dir])

for subid in subids:
    sub_file = os.path.join('ds000030_R1.0.4','derivatives',
                            'fmriprep', subid,'func',
                            '{}_task-rest_bold_space-T1w_preproc.nii.gz'.format(subid))
    if not os.path.exists(os.path.join(src_data_dir,sub_file)):
        if not os.path.exists(os.path.join(src_data_dir, 
                                           os.path.basename(func_webtgz))):
            run(['wget', '-P', src_data_dir, func_webtgz])

        run(['unzip', 
             os.path.join(src_data_dir, 
                          os.path.basename(func_webtgz)), 
             sub_file,
             '-d', src_data_dir])

for subid in subids:
    sub_file = os.path.join('ds000030_R1.0.4','derivatives',
                            'fmriprep', subid,'func',
                            '{}_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'.format(subid))
    if not os.path.exists(os.path.join(src_data_dir,sub_file)):
        if not os.path.exists(os.path.join(src_data_dir, 
                                           os.path.basename(func_webtgz))):
            run(['wget', '-P', src_data_dir, func_webtgz])

        run(['unzip', 
             os.path.join(src_data_dir, 
                          os.path.basename(func_webtgz)), 
             sub_file,
             '-d', src_data_dir])

In [17]:
run(['ciftify_recon_all', '--resample-to-T1w32k', '--hcp-data-dir', hcp_data_dir,
    '--fs-subjects-dir', fs_subjects_dir, 
    subids[0]])

Running: ciftify_recon_all --resample-to-T1w32k --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --fs-subjects-dir /home/edickie/Documents/ciftify_tests/src_data/ds000030_R1.0.4/derivatives/freesurfer sub-50005
Running: ciftify_recon_all --resample-to-T1w32k --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --fs-subjects-dir /home/edickie/Documents/ciftify_tests/src_data/ds000030_R1.0.4/derivatives/freesurfer sub-50005


0

In [18]:
run(['ciftify_recon_all', '--hcp-data-dir', hcp_data_dir,
    '--fs-subjects-dir', fs_subjects_dir, 
    subids[1]])

Running: ciftify_recon_all --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --fs-subjects-dir /home/edickie/Documents/ciftify_tests/src_data/ds000030_R1.0.4/derivatives/freesurfer sub-50007
Running: ciftify_recon_all --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --fs-subjects-dir /home/edickie/Documents/ciftify_tests/src_data/ds000030_R1.0.4/derivatives/freesurfer sub-50007


0

In [19]:
for subid in subids:
    run(['cifti_vis_recon_all', 'snaps', '--hcp-data-dir',hcp_data_dir, subid])
run(['cifti_vis_recon_all', 'index', '--hcp-data-dir', hcp_data_dir])

Running: cifti_vis_recon_all snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp sub-50005
Running: cifti_vis_recon_all snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp sub-50005
Running: cifti_vis_recon_all snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp sub-50007
Running: cifti_vis_recon_all snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp sub-50007
Running: cifti_vis_recon_all index --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp
Running: cifti_vis_recon_all index --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp


0

In [ ]:
# for subid in ['sub-50004','sub-50006', 'sub-50008']:
#     run(['ciftify_recon_all', '--hcp-data-dir', hcp_data_dir,
#         '--fs-subjects-dir', fs_subjects_dir, 
#         subid])

In [ ]:
# for subid in ['sub-50004','sub-50006', 'sub-50008']:
#     run(['cifti_vis_recon_all', 'snaps', '--hcp-data-dir',hcp_data_dir, subid])
# run(['cifti_vis_recon_all', 'index', '--hcp-data-dir', hcp_data_dir])

# Running ciftify_subject_fmri 

In [38]:
subid=subids[0]
native_func = os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subid,'func',
                 '{}_task-rest_bold_space-native_preproc.nii.gz'.format(subid))
mat_file = os.path.join(src_data_dir, 'ds000030_R1.0.4','derivatives',
                            'fmriprep', subid,'func',
                            '{}_task-rest_bold_T1_to_EPI.mat'.format(subid))

with open(mat_file, "w") as text_file:
    text_file.write('''1.03  -0.015  0.0025  -15.0
0.014  1.01  -0.005  -11.9
-0.007 0.01  0.99  2
0  0  0  1
''')

t1_func = os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subid,'func',
                 '{}_task-rest_bold_space-T1w_preproc.nii.gz'.format(subid))
if not os.path.exists(native_func):
    run(['flirt', '-in', t1_func, '-ref', t1_func, 
         '-out', native_func, '-init', mat_file, '-applyxfm'])

run(['ciftify_subject_fmri', '--SmoothingFWHM', '12', 
    '--hcp-data-dir', hcp_data_dir, 
    native_func, subid, 'rest_test1'])

Running: ciftify_subject_fmri --SmoothingFWHM 12 --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50005/func/sub-50005_task-rest_bold_space-native_preproc.nii.gz sub-50005 rest_test1
Running: ciftify_subject_fmri --SmoothingFWHM 12 --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50005/func/sub-50005_task-rest_bold_space-native_preproc.nii.gz sub-50005 rest_test1


0

In [39]:
subid=subids[0]
run(['cifti_vis_fmri', 'snaps', '--SmoothingFWHM', '12', '--hcp-data-dir',hcp_data_dir, 'rest_test1', subid])

Running: cifti_vis_fmri snaps --SmoothingFWHM 12 --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp rest_test1 sub-50005
Running: cifti_vis_fmri snaps --SmoothingFWHM 12 --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp rest_test1 sub-50005


0

In [40]:
subid=subids[0]
native_func_mean = os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subid,'func',
                 '{}_task-rest_bold_space-native_mean.nii.gz'.format(subid))

run(['fslmaths', native_func, '-Tmean', native_func_mean])

run(['ciftify_subject_fmri', '--hcp-data-dir', hcp_data_dir, 
    '--FLIRT-template', native_func_mean, native_func, subid, 'rest_test2'])


Running: fslmaths /home/edickie/Documents/ciftify_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50005/func/sub-50005_task-rest_bold_space-native_preproc.nii.gz -Tmean /home/edickie/Documents/ciftify_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50005/func/sub-50005_task-rest_bold_space-native_mean.nii.gz
Running: fslmaths /home/edickie/Documents/ciftify_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50005/func/sub-50005_task-rest_bold_space-native_preproc.nii.gz -Tmean /home/edickie/Documents/ciftify_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50005/func/sub-50005_task-rest_bold_space-native_mean.nii.gz
Running: ciftify_subject_fmri --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --FLIRT-template /home/edickie/Documents/ciftify_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50005/func/sub-50005_task-rest_bold_space-native_mean.nii.gz /home/edickie/Documents/ciftify_tests/src_data/ds000030_R1.0.4/derivatives/fmrip

0

In [41]:
subid=subids[0]
run(['cifti_vis_fmri', 'snaps', '--smooth-conn', '12', '--hcp-data-dir',hcp_data_dir, 'rest_test2', subid])

Running: cifti_vis_fmri snaps --smooth-conn 12 --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp rest_test2 sub-50005
Running: cifti_vis_fmri snaps --smooth-conn 12 --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp rest_test2 sub-50005


0

In [51]:

subid = subids[1]
run(['ciftify_subject_fmri', '--SmoothingFWHM', '8', '--FLIRT-dof', '30',
    '--hcp-data-dir', hcp_data_dir, 
     '--OutputSurfDiagnostics',
    os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subid,'func',
                 '{}_task-rest_bold_space-T1w_preproc.nii.gz'.format(subid)),
    subid, 'rest_test_dof30'])

Running: ciftify_subject_fmri --SmoothingFWHM 8 --FLIRT-dof 30 --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --OutputSurfDiagnostics /home/edickie/Documents/ciftify_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50007/func/sub-50007_task-rest_bold_space-T1w_preproc.nii.gz sub-50007 rest_test_dof30
Running: ciftify_subject_fmri --SmoothingFWHM 8 --FLIRT-dof 30 --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --OutputSurfDiagnostics /home/edickie/Documents/ciftify_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50007/func/sub-50007_task-rest_bold_space-T1w_preproc.nii.gz sub-50007 rest_test_dof30


0

In [52]:
subid = subids[1]
run(['cifti_vis_fmri', 'snaps', '--hcp-data-dir',hcp_data_dir, 'rest_test_dof30', subid])

Running: cifti_vis_fmri snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp rest_test_dof30 sub-50007
Running: cifti_vis_fmri snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp rest_test_dof30 sub-50007


0

In [44]:
subid = subids[1]
run(['ciftify_subject_fmri', '--already-in-MNI', '--SmoothingFWHM', '8', 
    '--hcp-data-dir', hcp_data_dir, 
    os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subid,'func',
                 '{}_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'.format(subid)),
    subid, 'rest_bad_transform'])

Running: ciftify_subject_fmri --already-in-MNI --SmoothingFWHM 8 --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50007/func/sub-50007_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz sub-50007 rest_bad_transform
Running: ciftify_subject_fmri --already-in-MNI --SmoothingFWHM 8 --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50007/func/sub-50007_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz sub-50007 rest_bad_transform


0

In [45]:
subid = subids[1]
run(['cifti_vis_fmri', 'snaps', '--hcp-data-dir',hcp_data_dir, 'rest_bad_transform', subid])

Running: cifti_vis_fmri snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp rest_bad_transform sub-50007
Running: cifti_vis_fmri snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp rest_bad_transform sub-50007


0

In [46]:
run(['cifti_vis_fmri', 'index', '--hcp-data-dir', hcp_data_dir])

Running: cifti_vis_fmri index --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp
Running: cifti_vis_fmri index --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp


0

In [53]:
from glob import glob
dtseries_files = glob(os.path.join(hcp_data_dir,'*',
                                       'MNINonLinear', 'Results',
                                       '*', '*Atlas_s*.dtseries.nii'))
run(['cifti_vis_RSN', 'cifti-snaps',  
         '--hcp-data-dir',hcp_data_dir, 
         '--colour-palette', 'PSYCH-NO-NONE',
         dtseries_files[0], 
         os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(dtseries_files[0])))))])
    
for dtseries in dtseries_files[1:]:
    run(['cifti_vis_RSN', 'cifti-snaps',  
         '--hcp-data-dir',hcp_data_dir, dtseries, 
         os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(dtseries)))))])
run(['cifti_vis_RSN', 'index', 
     '--hcp-data-dir', hcp_data_dir])

Running: cifti_vis_RSN cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --colour-palette PSYCH-NO-NONE /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/Results/rest_test_dof30/rest_test_dof30_Atlas_s0.dtseries.nii sub-50007
Running: cifti_vis_RSN cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --colour-palette PSYCH-NO-NONE /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/Results/rest_test_dof30/rest_test_dof30_Atlas_s0.dtseries.nii sub-50007
Running: cifti_vis_RSN cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/Results/rest_test_dof30/rest_test_dof30_Atlas_s8.dtseries.nii sub-50007
Running: cifti_vis_RSN cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/h

0

In [54]:
run(['cifti_vis_RSN', 'index', '--hcp-data-dir', hcp_data_dir])

Running: cifti_vis_RSN index --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp
Running: cifti_vis_RSN index --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp


0

In [173]:
for dtseries in dtseries_files:
    run_seed_corr_fmri_test(dtseries, hcp_data_dir)

Running: wb_command -volume-label-to-roi /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/wmparc.nii.gz /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50007_RIGHT-PUTAMEN_vol.nii.gz -name RIGHT-PUTAMEN
Running: wb_command -volume-label-to-roi /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/wmparc.nii.gz /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50007_RIGHT-PUTAMEN_vol.nii.gz -name RIGHT-PUTAMEN
Running: ciftify_seed_corr /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/Results/rest_test_dof30/rest_test_dof30_Atlas_s0.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50007_RIGHT-PUTAMEN_vol.nii.gz
Running: ciftify_seed_corr /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/Results/rest_test_dof30/rest_test_dof30_Atlas_s0.dtseries.nii /home/edickie/Documents/ciftify_te

Running: ciftify_seed_corr /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_sub-50005_RIGHT-PUTAMEN_vol.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_sub-50005_RIGHT-PUTAMEN_vol.dscalar.nii


In [75]:
run(['cifti_vis_RSN', 'nifti-snaps', '--debug', 
         '--hcp-data-dir',hcp_data_dir, 
         '--qcdir', os.path.join(hcp_data_dir, 'RSN_from_nii'),
         '--colour-palette', 'PSYCH-NO-NONE',
     
        os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subids[0],'func',
                 '{}_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'.format(subids[0])),
        subids[0]])

run(['cifti_vis_RSN', 'nifti-snaps',  
         '--hcp-data-dir',hcp_data_dir, 
         '--qcdir', os.path.join(hcp_data_dir, 'RSN_from_nii'),
        os.path.join(src_data_dir,'ds000030_R1.0.4','derivatives',
                 'fmriprep', subids[1],'func',
                 '{}_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'.format(subids[1])),
        subids[1]])
    
run(['cifti_vis_RSN', 'index', 
     '--qcdir', os.path.join(hcp_data_dir, 'RSN_from_nii'),
     '--hcp-data-dir', hcp_data_dir])

Running: cifti_vis_RSN nifti-snaps --debug --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --qcdir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/RSN_from_nii --colour-palette PSYCH-NO-NONE /home/edickie/Documents/ciftify_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50005/func/sub-50005_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz sub-50005
Running: cifti_vis_RSN nifti-snaps --debug --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --qcdir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/RSN_from_nii --colour-palette PSYCH-NO-NONE /home/edickie/Documents/ciftify_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50005/func/sub-50005_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz sub-50005
[cifti_vis_RSN] DEBUG: {'--colour-palette': 'PSYCH-NO-NONE',
 '--debug': True,
 '--hcp-data-dir': '/home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp',
 '--help': False,
 '--qcdir': '/home/edickie

[cifti_vis_RSN] DEBUG: {'--colour-palette': 'PSYCH-NO-NONE',
 '--debug': True,
 '--hcp-data-dir': '/home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp',
 '--help': False,
 '--qcdir': '/home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/RSN_from_nii',
 '--subjects-filter': None,
 '--subjects-list': None,
 '--verbose': False,
 '<func.dtseries.nii>': None,
 '<func.nii.gz>': '/home/edickie/Documents/ciftify_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50005/func/sub-50005_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz',
 '<subject>': 'sub-50005',
 'cifti-snaps': False,
 'index': False,
 'nifti-snaps': True}
[ciftify.utils] INFO: Running: mkdir -p /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/RSN_from_nii/sub-50005_sub-50005_task-rest_bold_space-MNI152NLin2009cAsym_preproc
[ciftify.utils] INFO: Running: mkdir -p /tmp/tmpa5n59cro/sub-50005
[ciftify.utils] INFO: Running: ciftify_vol_result --resample-nifti --hcp-data-dir /home/edickie/Documents/ciftify

Running: cifti_vis_RSN nifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --qcdir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/RSN_from_nii /home/edickie/Documents/ciftify_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50007/func/sub-50007_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz sub-50007
Running: cifti_vis_RSN nifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --qcdir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/RSN_from_nii /home/edickie/Documents/ciftify_tests/src_data/ds000030_R1.0.4/derivatives/fmriprep/sub-50007/func/sub-50007_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz sub-50007
Running: cifti_vis_RSN index --qcdir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/RSN_from_nii --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp
Running: cifti_vis_RSN index --qcdir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/RSN_fro

0

In [76]:
groupmask_cmd = ['ciftify_groupmask', os.path.join(hcp_data_dir, 'groupmask.dscalar.nii')]
groupmask_cmd.extend(dtseries_files)

run(groupmask_cmd)     

Running: ciftify_groupmask /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/groupmask.dscalar.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/Results/rest_test_dof30/rest_test_dof30_Atlas_s0.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/Results/rest_test_dof30/rest_test_dof30_Atlas_s8.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/Results/rest_bad_transform/rest_bad_transform_Atlas_s8.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/Results/rest_bad_transform/rest_bad_transform_Atlas_s0.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test2/rest_test2_Atlas_s0.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-

0

# Running PINT and related functions

In [58]:
subid = subids[0]
run(['mkdir', '-p', os.path.join(new_outputs, 'PINT', subid)])

run(['ciftify_PINT_vertices', '--pcorr', 
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test1', 'rest_test1_Atlas_s12.dtseries.nii'),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(ciftify.config.find_ciftify_global(), 'PINT', 'Yeo7_2011_80verts.csv'),
     os.path.join(new_outputs, 'PINT', subid, subid)])


Running: mkdir -p /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/sub-50005
Running: mkdir -p /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/sub-50005
Running: ciftify_PINT_vertices --pcorr /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness.32k_fs_LR.surf.gii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.R.midthickness.32k_fs_LR.surf.gii /home/edickie/code/ciftify/ciftify/data/PINT/Yeo7_2011_80verts.csv /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/sub-50005/sub-50005
Running: ciftify_PINT_vertices --pcorr /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/su

0

In [59]:
subid = subids[0]
run(['cifti_vis_PINT', 'snaps', '--hcp-data-dir',  hcp_data_dir,
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test1', 'rest_test1_Atlas_s12.dtseries.nii'),
     subid,
     os.path.join(new_outputs, 'PINT', subid, '{}_summary.csv'.format(subid))])

Running: cifti_vis_PINT snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii sub-50005 /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/sub-50005/sub-50005_summary.csv
Running: cifti_vis_PINT snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii sub-50005 /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/sub-50005/sub-50005_summary.csv


0

In [60]:
subid = subids[1]
run(['mkdir', '-p', os.path.join(new_outputs, 'PINT', subid)])

run(['ciftify_PINT_vertices',
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test_dof30', 'rest_test_dof30_Atlas_s8.dtseries.nii'),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(ciftify.config.find_ciftify_global(), 'PINT', 'Yeo7_2011_80verts.csv'),
     os.path.join(new_outputs, 'PINT', subid, subid)])

Running: mkdir -p /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/sub-50007
Running: mkdir -p /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/sub-50007
Running: ciftify_PINT_vertices /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/Results/rest_test_dof30/rest_test_dof30_Atlas_s8.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/fsaverage_LR32k/sub-50007.L.midthickness.32k_fs_LR.surf.gii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/fsaverage_LR32k/sub-50007.R.midthickness.32k_fs_LR.surf.gii /home/edickie/code/ciftify/ciftify/data/PINT/Yeo7_2011_80verts.csv /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/sub-50007/sub-50007
Running: ciftify_PINT_vertices /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/Results/rest_test_dof30/rest_test_dof30_Atlas_s8.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/

0

In [61]:
run(['cifti_vis_PINT', 'snaps', '--hcp-data-dir',  hcp_data_dir,
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test_dof30', 'rest_test_dof30_Atlas_s8.dtseries.nii'),
     subid,
     os.path.join(new_outputs, 'PINT', subid, '{}_summary.csv'.format(subid))])

Running: cifti_vis_PINT snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/Results/rest_test_dof30/rest_test_dof30_Atlas_s8.dtseries.nii sub-50007 /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/sub-50007/sub-50007_summary.csv
Running: cifti_vis_PINT snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/Results/rest_test_dof30/rest_test_dof30_Atlas_s8.dtseries.nii sub-50007 /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/sub-50007/sub-50007_summary.csv


0

In [62]:
subid = subids[1]
run(['mkdir', '-p', os.path.join(new_outputs, 'PINT', subid)])

run(['ciftify_PINT_vertices', '--outputall',
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test_dof30', 'rest_test_dof30_Atlas_s8.dtseries.nii'),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(ciftify.config.find_ciftify_global(), 'PINT', 'Yeo7_2011_80verts.csv'),
     os.path.join(new_outputs, 'PINT', subid, '{}_all'.format(subid))])

Running: mkdir -p /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/sub-50007
Running: mkdir -p /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/sub-50007
Running: ciftify_PINT_vertices --outputall /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/Results/rest_test_dof30/rest_test_dof30_Atlas_s8.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/fsaverage_LR32k/sub-50007.L.midthickness.32k_fs_LR.surf.gii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/fsaverage_LR32k/sub-50007.R.midthickness.32k_fs_LR.surf.gii /home/edickie/code/ciftify/ciftify/data/PINT/Yeo7_2011_80verts.csv /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/sub-50007/sub-50007_all
Running: ciftify_PINT_vertices --outputall /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/Results/rest_test_dof30/rest_test_dof30_Atlas_s8.dtseries.nii /home/edickie/Documents/cifti

0

In [63]:
run(['cifti_vis_PINT', 'snaps', '--hcp-data-dir',  hcp_data_dir,
     os.path.join(hcp_data_dir, subid,'MNINonLinear', 'Results',
                  'rest_test_dof30', 'rest_test_dof30_Atlas_s8.dtseries.nii'),
     subid,
     os.path.join(new_outputs, 'PINT', subid, '{}_all_summary.csv'.format(subid))])

Running: cifti_vis_PINT snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/Results/rest_test_dof30/rest_test_dof30_Atlas_s8.dtseries.nii sub-50007 /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/sub-50007/sub-50007_all_summary.csv
Running: cifti_vis_PINT snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50007/MNINonLinear/Results/rest_test_dof30/rest_test_dof30_Atlas_s8.dtseries.nii sub-50007 /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/sub-50007/sub-50007_all_summary.csv


0

In [64]:
run(['cifti_vis_PINT', 'index', '--hcp-data-dir',  hcp_data_dir])

Running: cifti_vis_PINT index --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp
Running: cifti_vis_PINT index --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp


0

In [65]:
summary_files = glob(os.path.join(new_outputs, 'PINT', '*','*_summary.csv'))
concat_cmd = ['ciftify_postPINT1_concat.py',  
           os.path.join(new_outputs, 'PINT', 'concatenated.csv')]
concat_cmd.extend(summary_files)
run(concat_cmd)

Running: ciftify_postPINT1_concat.py /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/concatenated.csv /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/sub-50007/sub-50007_summary.csv /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/sub-50007/sub-50007_all_summary.csv /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/sub-50005/sub-50005_summary.csv
Running: ciftify_postPINT1_concat.py /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/concatenated.csv /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/sub-50007/sub-50007_summary.csv /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/sub-50007/sub-50007_all_summary.csv /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/sub-50005/sub-50005_summary.csv


0

In [66]:
run(['ciftify_postPINT2_sub2sub.py', os.path.join(new_outputs, 'PINT', 'concatenated.csv'),
     os.path.join(new_outputs, 'PINT', 'ivertex_distances.csv')])

Running: ciftify_postPINT2_sub2sub.py /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/concatenated.csv /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/ivertex_distances.csv
Running: ciftify_postPINT2_sub2sub.py /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/concatenated.csv /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/ivertex_distances.csv


0

In [67]:
run(['ciftify_postPINT2_sub2sub.py', '--roiidx', '14',
     os.path.join(new_outputs, 'PINT', 'concatenated.csv'),
     os.path.join(new_outputs, 'PINT', 'ivertex_distances_roiidx14.csv')])

Running: ciftify_postPINT2_sub2sub.py --roiidx 14 /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/concatenated.csv /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/ivertex_distances_roiidx14.csv
Running: ciftify_postPINT2_sub2sub.py --roiidx 14 /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/concatenated.csv /home/edickie/Documents/ciftify_tests/run_2017-09-08/PINT/ivertex_distances_roiidx14.csv


0

# Running ciftify_surface_rois

In [77]:
vertices1_csv = os.path.join(src_data_dir, 'vertices1.csv')

with open(vertices1_csv, "w") as text_file:
    text_file.write('''hemi,vertex
L,11801
L,26245
L,26235
L,26257
L,13356
L,289
L,13336
L,13337
L,26269
L,13323
L,26204
L,26214
L,13326
L,13085
L,13310
L,13281
L,13394
L,13395
L,26263
L,26265
L,13343
L,77
L,13273
L,13342
L,26271
L,11804
L,13322
L,13369
L,13353
L,26268
L,26201
L,26269
L,68
L,13391
''')

subid = subids[0]
run(['mkdir', '-p', os.path.join(new_outputs, 'rois')])
run(['ciftify_surface_rois', vertices1_csv, '10', '--gaussian',
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)), 
     os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii')])

Running: mkdir -p /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois
Running: mkdir -p /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois
Running: ciftify_surface_rois /home/edickie/Documents/ciftify_tests/src_data/vertices1.csv 10 --gaussian /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness.32k_fs_LR.surf.gii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.R.midthickness.32k_fs_LR.surf.gii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/gaussian_roi.dscalar.nii
Running: ciftify_surface_rois /home/edickie/Documents/ciftify_tests/src_data/vertices1.csv 10 --gaussian /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness.32k_fs_LR.surf.gii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.R.midthickness.32k_fs_LR.surf

0

In [78]:
subid = subids[0]
run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii'),
     subid, 
     'gaussian_roi'])

Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/gaussian_roi.dscalar.nii sub-50005 gaussian_roi
Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/gaussian_roi.dscalar.nii sub-50005 gaussian_roi


0

In [79]:
run(['ciftify_peaktable', '--max-threshold', '0.05',      
    '--left-surface', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
    '--right-surface', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)),
    os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii')])

Running: ciftify_peaktable --max-threshold 0.05 --left-surface /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness.32k_fs_LR.surf.gii --right-surface /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.R.midthickness.32k_fs_LR.surf.gii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/gaussian_roi.dscalar.nii
Running: ciftify_peaktable --max-threshold 0.05 --left-surface /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness.32k_fs_LR.surf.gii --right-surface /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.R.midthickness.32k_fs_LR.surf.gii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/gaussian_roi.dscalar.nii


0

In [80]:
vertices2_csv = os.path.join(src_data_dir, 'vertices1.csv')

with open(vertices2_csv, "w") as text_file:
    text_file.write('''hemi,vertex
R,2379
R,2423
R,2423
R,2629
R,29290
R,29290
R,1794
R,29199
R,1788
R,2380
R,2288
R,29320
R,29274
R,29272
R,29331
R,29356
R,2510
R,29345
R,2506
R,2506
R,1790
R,29305
R,2630
R,2551
R,2334
R,2334
R,29306
R,2551
R,2422
R,29256
R,2468
R,29332
R,2425
R,29356
''')
    
run(['mkdir', '-p', os.path.join(new_outputs, 'rois')])
run(['ciftify_surface_rois', vertices1_csv, '6', '--probmap',
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
     os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)), 
     os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii')])

Running: mkdir -p /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois
Running: mkdir -p /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois
Running: ciftify_surface_rois /home/edickie/Documents/ciftify_tests/src_data/vertices1.csv 6 --probmap /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness.32k_fs_LR.surf.gii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.R.midthickness.32k_fs_LR.surf.gii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/probmap_roi.dscalar.nii
Running: ciftify_surface_rois /home/edickie/Documents/ciftify_tests/src_data/vertices1.csv 6 --probmap /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness.32k_fs_LR.surf.gii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.R.midthickness.32k_fs_LR.surf.gii 

0

In [81]:
subid = subids[0]
run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii'),
     subid, 
     'probmap_roi'])

Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/probmap_roi.dscalar.nii sub-50005 probmap_roi
Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/probmap_roi.dscalar.nii sub-50005 probmap_roi


0

In [82]:
run(['ciftify_peaktable', '--max-threshold', '0.05',      
    '--left-surface', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
    '--right-surface', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)), 
   os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii')])

Running: ciftify_peaktable --max-threshold 0.05 --left-surface /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness.32k_fs_LR.surf.gii --right-surface /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.R.midthickness.32k_fs_LR.surf.gii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/probmap_roi.dscalar.nii
Running: ciftify_peaktable --max-threshold 0.05 --left-surface /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness.32k_fs_LR.surf.gii --right-surface /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.R.midthickness.32k_fs_LR.surf.gii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/probmap_roi.dscalar.nii


0

In [83]:
for hemi in ['L','R']:
    run(['wb_command', '-surface-vertex-areas',
         os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.{}.midthickness.32k_fs_LR.surf.gii'.format(subid, hemi)),
         os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.{}.midthickness_va.32k_fs_LR.shape.gii'.format(subid, hemi))])

run(['ciftify_peaktable', 
    '--outputbase', os.path.join(new_outputs, 'rois', 'probmap_roi_withva'),
    '--max-threshold', '0.05',      
    '--left-surface', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness.32k_fs_LR.surf.gii'.format(subid)),
    '--right-surface', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness.32k_fs_LR.surf.gii'.format(subid)), 
    '--left-surf-area', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.L.midthickness_va.32k_fs_LR.shape.gii'.format(subid)),
    '--right-surf-area', os.path.join(hcp_data_dir, subid,'MNINonLinear','fsaverage_LR32k',
                  '{}.R.midthickness_va.32k_fs_LR.shape.gii'.format(subid)), 
   os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii')])

Running: wb_command -surface-vertex-areas /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness.32k_fs_LR.surf.gii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness_va.32k_fs_LR.shape.gii
Running: wb_command -surface-vertex-areas /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness.32k_fs_LR.surf.gii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.L.midthickness_va.32k_fs_LR.shape.gii
Running: wb_command -surface-vertex-areas /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.R.midthickness.32k_fs_LR.surf.gii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.R.midthickness_va.32k_fs_LR.shape.gii
Running: wb_command -s

0

In [84]:
run(['ciftify_surface_rois', '--debug',
      os.path.join(ciftify.config.find_ciftify_global(), 'PINT', 'Yeo7_2011_80verts.csv'),
      '6', '--vertex-col', 'tvertex', '--labels-col', 'NETWORK', '--overlap-logic', 'EXCLUDE',
      os.path.join(ciftify.config.find_HCP_S1200_GroupAvg(),
                'S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii'),
      os.path.join(ciftify.config.find_HCP_S1200_GroupAvg(),
                'S1200.R.midthickness_MSMAll.32k_fs_LR.surf.gii'),
      os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii')])

Running: ciftify_surface_rois --debug /home/edickie/code/ciftify/ciftify/data/PINT/Yeo7_2011_80verts.csv 6 --vertex-col tvertex --labels-col NETWORK --overlap-logic EXCLUDE /home/edickie/code/ciftify/ciftify/data/HCP_S1200_GroupAvg_v1/S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii /home/edickie/code/ciftify/ciftify/data/HCP_S1200_GroupAvg_v1/S1200.R.midthickness_MSMAll.32k_fs_LR.surf.gii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/tvertex.dscalar.nii
Running: ciftify_surface_rois --debug /home/edickie/code/ciftify/ciftify/data/PINT/Yeo7_2011_80verts.csv 6 --vertex-col tvertex --labels-col NETWORK --overlap-logic EXCLUDE /home/edickie/code/ciftify/ciftify/data/HCP_S1200_GroupAvg_v1/S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii /home/edickie/code/ciftify/ciftify/data/HCP_S1200_GroupAvg_v1/S1200.R.midthickness_MSMAll.32k_fs_LR.surf.gii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/tvertex.dscalar.nii


0

In [85]:
subid = subids[0]
run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii'),
     subid, 
     'tvertex'])

Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/tvertex.dscalar.nii sub-50005 tvertex
Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/tvertex.dscalar.nii sub-50005 tvertex


0

# Running ciftify_seed_corr 

In [89]:
subid = subids[0]
smoothing = 12
func_vol = os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'Results', 'rest_test1', 'rest_test1.nii.gz')
func_cifti_sm0 = os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'Results', 'rest_test1', 'rest_test1_Atlas_s0.dtseries.nii')
func_cifti_smoothed = os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'Results', 'rest_test1', 'rest_test1_Atlas_s{}.dtseries.nii'.format(smoothing))
atlas_vol =  os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'wmparc.nii.gz')

seed_corr_dir = os.path.join(new_outputs, 'ciftify_seed_corr')
if not os.path.exists(seed_corr_dir):
    run(['mkdir', '-p', seed_corr_dir])
struct = 'RIGHT-PUTAMEN'
putamen_vol_seed_mask = os.path.join(seed_corr_dir, '{}_{}_vol.nii.gz'.format(subid, struct))
run(['wb_command', '-volume-label-to-roi', atlas_vol, putamen_vol_seed_mask, '-name', struct])



Running: mkdir -p /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr
Running: mkdir -p /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr
Running: wb_command -volume-label-to-roi /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/wmparc.nii.gz /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz -name RIGHT-PUTAMEN
Running: wb_command -volume-label-to-roi /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/wmparc.nii.gz /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz -name RIGHT-PUTAMEN


0

In [91]:
run(['ciftify_seed_corr', func_vol, putamen_vol_seed_mask])
run(['cifti_vis_map', 'nifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     seed_corr_default_out(func_vol, putamen_vol_seed_mask),
     subid, 
     '{}_{}_niftitonifti_unmasked'.format(subid, struct)])

Running: cifti_vis_map nifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_sub-50005_RIGHT-PUTAMEN_vol.nii.gz sub-50005 sub-50005_RIGHT-PUTAMEN_niftitonifti_unmasked
Running: cifti_vis_map nifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_sub-50005_RIGHT-PUTAMEN_vol.nii.gz sub-50005 sub-50005_RIGHT-PUTAMEN_niftitonifti_unmasked


0

In [92]:
run(['ciftify_seed_corr',
     '--outputname', os.path.join(seed_corr_dir, '{}_{}_niftitonifti_masked.nii.gz'.format(subid, struct)),
     '--mask', os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'brainmask_fs.nii.gz'),
     func_vol, putamen_vol_seed_mask])

run(['cifti_vis_map', 'nifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     os.path.join(seed_corr_dir, '{}_{}_niftitonifti_masked.nii.gz'.format(subid, struct)),
     subid, 
     '{}_{}_niftitonifti_masked'.format(subid, struct)])

Running: ciftify_seed_corr --outputname /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_niftitonifti_masked.nii.gz --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/brainmask_fs.nii.gz /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1.nii.gz /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz
Running: ciftify_seed_corr --outputname /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_niftitonifti_masked.nii.gz --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/brainmask_fs.nii.gz /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1.nii.gz /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz


0

In [94]:
run(['ciftify_seed_corr', func_cifti_smoothed, putamen_vol_seed_mask])

Running: ciftify_seed_corr /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz
Running: ciftify_seed_corr /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz


0

In [95]:
run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', 
     hcp_data_dir, 
     seed_corr_default_out(func_cifti_smoothed, putamen_vol_seed_mask),
     subid, 
     '{}_{}_niftitocifti_unmasked'.format(subid, struct)])

Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_sub-50005_RIGHT-PUTAMEN_vol.dscalar.nii sub-50005 sub-50005_RIGHT-PUTAMEN_niftitocifti_unmasked
Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_sub-50005_RIGHT-PUTAMEN_vol.dscalar.nii sub-50005 sub-50005_RIGHT-PUTAMEN_niftitocifti_unmasked


0

In [96]:
run_seedcorr_peaktable(seed_corr_default_out(func_cifti_smoothed, putamen_vol_seed_mask))

Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_sub-50005_RIGHT-PUTAMEN_vol.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_sub-50005_RIGHT-PUTAMEN_vol.dscalar.nii


In [97]:
cifti_mask = os.path.join(seed_corr_dir, '{}_func_mask.dscalar.nii'.format(subid))
run(['wb_command', '-cifti-math', "'(x > 0)'", cifti_mask,
    '-var', 'x', func_cifti_sm0, '-select', '1', '1'])
run(['ciftify_seed_corr',
     '--outputname', os.path.join(seed_corr_dir, '{}_{}_niftitocifti_masked.dscalar.nii'.format(subid, struct)),
     '--mask', cifti_mask,
     func_cifti_smoothed, putamen_vol_seed_mask])

Running: wb_command -cifti-math '(x > 0)' /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii -var x /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii -select 1 1
Running: wb_command -cifti-math '(x > 0)' /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii -var x /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii -select 1 1
parsed '(x > 0)' as '(x>0)'

parsed '(x > 0)' as '(x>0)'

Running: ciftify_seed_corr --outputname /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_niftitocifti_masked.dscalar.nii --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINon

0

In [98]:
subid = subids[0]
run(['cifti_vis_map', 'cifti-snaps', '--hcp-data-dir', hcp_data_dir, 
     os.path.join(seed_corr_dir, '{}_{}_niftitocifti_masked.dscalar.nii'.format(subid, struct)),
     subid, 
     '{}_{}_niftitocifti_masked'.format(subid, struct)])
run_seedcorr_peaktable(os.path.join(seed_corr_dir, '{}_{}_niftitocifti_masked.dscalar.nii'.format(subid, struct)))

Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_niftitocifti_masked.dscalar.nii sub-50005 sub-50005_RIGHT-PUTAMEN_niftitocifti_masked
Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_niftitocifti_masked.dscalar.nii sub-50005 sub-50005_RIGHT-PUTAMEN_niftitocifti_masked
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_niftitocifti_masked.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_niftitocifti_masked.dscala

In [99]:
result_map = seed_corr_default_out(func_cifti_smoothed, 
                                   os.path.join(seed_corr_dir, '{}_{}_cifti.dscalar.nii'.format(subid, struct)))
result_prefix = '{}_{}_ciftitocifti_unmasked'.format(subid, struct)
result_type = 'cifti'

In [100]:
run(['wb_command', '-cifti-create-dense-from-template',
     func_cifti_sm0,
     os.path.join(seed_corr_dir, 
                                 '{}_{}_cifti.dscalar.nii'.format(subid, struct)),
     '-volume-all', putamen_vol_seed_mask])

run(['ciftify_seed_corr', func_cifti_smoothed, 
    os.path.join(seed_corr_dir, '{}_{}_cifti.dscalar.nii'.format(subid, struct))])

Running: wb_command -cifti-create-dense-from-template /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_cifti.dscalar.nii -volume-all /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz
Running: wb_command -cifti-create-dense-from-template /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_cifti.dscalar.nii -volume-all /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_RIGHT-PUTAMEN_vol.nii.gz
Running: ciftify_seed_corr /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/

0

In [101]:
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_sub-50005_RIGHT-PUTAMEN_cifti.dscalar.nii sub-50005 sub-50005_RIGHT-PUTAMEN_ciftitocifti_unmasked
Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_sub-50005_RIGHT-PUTAMEN_cifti.dscalar.nii sub-50005 sub-50005_RIGHT-PUTAMEN_ciftitocifti_unmasked
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_sub-50005_RIGHT-PUTAMEN_cifti.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edicki

In [102]:
result_map = os.path.join(seed_corr_dir, '{}_{}_ciftitocifti_30TRs.dscalar.nii'.format(subid, struct))
result_type = 'cifti'
result_prefix = '{}_{}_ciftitocifti_30TRs'.format(subid, struct)

TR_file = os.path.join(new_outputs, 'rois','TR_file.txt')
with open(TR_file, "w") as text_file:
    text_file.write('''1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30''')

run(['ciftify_seed_corr',
     '--outputname', result_map,
     '--use-TRs', TR_file,
     func_cifti_smoothed, putamen_vol_seed_mask])
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

In [104]:
result_map = os.path.join(seed_corr_dir, '{}_{}_ciftitocifti_masked.dscalar.nii'.format(subid, struct))
result_type = 'cifti'
result_prefix = '{}_{}_ciftitocifti_masked'.format(subid, struct)

run(['ciftify_seed_corr',     
     '--outputname', result_map,
     '--mask', cifti_mask,
     func_cifti_smoothed, 
    os.path.join(seed_corr_dir, '{}_{}_cifti.dscalar.nii'.format(subid, struct))])

run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

In [107]:
result_map = seed_corr_default_out(func_cifti_smoothed, os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii'))
result_type = 'cifti'
result_prefix = '{}_gaussian_ciftitocifti_unmasked'.format(subid)


In [108]:
run(['ciftify_seed_corr', '--weighted', func_cifti_smoothed, 
     os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii')])

Running: ciftify_seed_corr --weighted /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/gaussian_roi.dscalar.nii
Running: ciftify_seed_corr --weighted /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/gaussian_roi.dscalar.nii


0

In [109]:
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_gaussian_roi.dscalar.nii sub-50005 sub-50005_gaussian_ciftitocifti_unmasked
Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_gaussian_roi.dscalar.nii sub-50005 sub-50005_gaussian_ciftitocifti_unmasked
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_gaussian_roi.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINon

In [110]:
result_map = os.path.join(seed_corr_dir, '{}_gaussian_ciftitocifti_masked.dscalar.nii'.format(subid))
result_type = 'cifti'
result_prefix = '{}_gaussian_ciftitocifti_masked'.format(subid)

In [111]:
run(['ciftify_seed_corr', '--weighted', 
     '--outputname', result_map,
     '--mask', cifti_mask,
     func_cifti_smoothed, 
     os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii')])

Running: ciftify_seed_corr --weighted --outputname /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_gaussian_ciftitocifti_masked.dscalar.nii --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/gaussian_roi.dscalar.nii
Running: ciftify_seed_corr --weighted --outputname /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_gaussian_ciftitocifti_masked.dscalar.nii --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/gaussian_roi.dscalar.nii


0

In [112]:
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_gaussian_ciftitocifti_masked.dscalar.nii sub-50005 sub-50005_gaussian_ciftitocifti_masked
Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_gaussian_ciftitocifti_masked.dscalar.nii sub-50005 sub-50005_gaussian_ciftitocifti_masked
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_gaussian_ciftitocifti_masked.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_gaussian_ciftitocifti_masked.dscalar.nii


In [113]:
L_gaussian_roi = os.path.join(new_outputs, 'rois', 'gaussian_L_roi.shape.gii')

result_map = seed_corr_default_out(func_cifti_smoothed, L_gaussian_roi)
result_type = 'cifti'
result_prefix = '{}_gaussian_giftitocifti_unmasked'.format(subid)


run(['wb_command', '-cifti-separate', 
     os.path.join(new_outputs, 'rois', 'gaussian_roi.dscalar.nii'),
     'COLUMN','-metric', 'CORTEX_LEFT', L_gaussian_roi])
run(['ciftify_seed_corr', '--weighted', '--hemi', 'L', 
     func_cifti_smoothed, 
     L_gaussian_roi])

Running: wb_command -cifti-separate /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/gaussian_roi.dscalar.nii COLUMN -metric CORTEX_LEFT /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/gaussian_L_roi.shape.gii
Running: wb_command -cifti-separate /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/gaussian_roi.dscalar.nii COLUMN -metric CORTEX_LEFT /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/gaussian_L_roi.shape.gii
Running: ciftify_seed_corr --weighted --hemi L /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/gaussian_L_roi.shape.gii
Running: ciftify_seed_corr --weighted --hemi L /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/gaussian_L_roi.shape.gii


0

In [114]:
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_gaussian_L_roi.dscalar.nii sub-50005 sub-50005_gaussian_giftitocifti_unmasked
Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_gaussian_L_roi.dscalar.nii sub-50005 sub-50005_gaussian_giftitocifti_unmasked
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_gaussian_L_roi.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/

In [115]:
result_map = os.path.join(seed_corr_dir, '{}_gaussian_giftitocifti_masked.dscalar.nii'.format(subid))
result_type = 'cifti'
result_prefix = '{}_gaussian_giftitocifti_masked.dscalar.nii'.format(subid)

In [116]:
run(['ciftify_seed_corr', '--weighted', '--hemi', 'L', 
     '--outputname', os.path.join(seed_corr_dir, '{}_gaussian_giftitocifti_masked.dscalar.nii'.format(subid)),
     '--mask', cifti_mask,
     func_cifti_smoothed, 
     L_gaussian_roi])

Running: ciftify_seed_corr --weighted --hemi L --outputname /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_gaussian_giftitocifti_masked.dscalar.nii --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/gaussian_L_roi.shape.gii
Running: ciftify_seed_corr --weighted --hemi L --outputname /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_gaussian_giftitocifti_masked.dscalar.nii --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/gaussian_

0

In [117]:
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_gaussian_giftitocifti_masked.dscalar.nii sub-50005 sub-50005_gaussian_giftitocifti_masked.dscalar.nii
Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_gaussian_giftitocifti_masked.dscalar.nii sub-50005 sub-50005_gaussian_giftitocifti_masked.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_gaussian_giftitocifti_masked.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_gaussian_giftitocifti_masked.dscalar.nii


In [118]:
result_map = seed_corr_default_out(func_cifti_smoothed, os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii'))
result_type = 'cifti'
result_prefix = '{}_promap_ciftitocifti_unmasked.dscalar.nii'.format(subid)

In [119]:
run(['ciftify_seed_corr', '--weighted', func_cifti_smoothed, 
     os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii')])

Running: ciftify_seed_corr --weighted /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/probmap_roi.dscalar.nii
Running: ciftify_seed_corr --weighted /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/probmap_roi.dscalar.nii


0

In [120]:
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_probmap_roi.dscalar.nii sub-50005 sub-50005_promap_ciftitocifti_unmasked.dscalar.nii
Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_probmap_roi.dscalar.nii sub-50005 sub-50005_promap_ciftitocifti_unmasked.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12_probmap_roi.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp

In [121]:
result_map = os.path.join(seed_corr_dir, '{}_promap_ciftitocifti_masked.dscalar.nii'.format(subid))
result_type = 'cifti'
result_prefix = '{}_promap_ciftitocifti_masked.dscalar.nii'.format(subid)

In [122]:
run(['ciftify_seed_corr', '--weighted', 
     '--outputname', result_map,
     '--mask', cifti_mask,
     func_cifti_smoothed, 
     os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii')])

Running: ciftify_seed_corr --weighted --outputname /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_promap_ciftitocifti_masked.dscalar.nii --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/probmap_roi.dscalar.nii
Running: ciftify_seed_corr --weighted --outputname /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_promap_ciftitocifti_masked.dscalar.nii --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/probmap_roi.dscalar.nii


0

In [123]:
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_promap_ciftitocifti_masked.dscalar.nii sub-50005 sub-50005_promap_ciftitocifti_masked.dscalar.nii
Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_promap_ciftitocifti_masked.dscalar.nii sub-50005 sub-50005_promap_ciftitocifti_masked.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_promap_ciftitocifti_masked.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_promap_ciftitocifti_masked.dscalar.nii


In [124]:
R_probmap_roi = os.path.join(new_outputs, 'rois', 'probmap_R_roi.shape.gii')
result_map = seed_corr_default_out(func_cifti_smoothed, R_probmap_roi) 
result_type = 'cifti'
result_prefix = '{}_promap_giftitocifti_unmasked.dscalar.nii'.format(subid)
                                   
run(['wb_command', '-cifti-separate', 
     os.path.join(new_outputs, 'rois', 'probmap_roi.dscalar.nii'),
     'COLUMN','-metric', 'CORTEX_RIGHT', R_probmap_roi])
run(['ciftify_seed_corr', '--weighted', '--hemi', 'R', func_cifti_smoothed, 
     R_probmap_roi])

Running: wb_command -cifti-separate /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/probmap_roi.dscalar.nii COLUMN -metric CORTEX_RIGHT /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/probmap_R_roi.shape.gii
Running: wb_command -cifti-separate /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/probmap_roi.dscalar.nii COLUMN -metric CORTEX_RIGHT /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/probmap_R_roi.shape.gii
Running: ciftify_seed_corr --weighted --hemi R /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/probmap_R_roi.shape.gii
Running: ciftify_seed_corr --weighted --hemi R /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/probmap_R_roi.shape.gii


0

In [125]:
result_map = os.path.join(seed_corr_dir, '{}_promap_giftitocifti_masked.dscalar.nii'.format(subid))
result_type = 'cifti'
result_prefix = '{}_promap_giftitocifti_masked.dscalar.nii'.format(subid)

In [126]:
run(['ciftify_seed_corr', '--weighted', '--hemi', 'R', 
     '--outputname', os.path.join(seed_corr_dir, '{}_promap_giftitocifti_masked.dscalar.nii'.format(subid)),
     '--mask', cifti_mask,    
     func_cifti_smoothed, 
     R_probmap_roi])

Running: ciftify_seed_corr --weighted --hemi R --outputname /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_promap_giftitocifti_masked.dscalar.nii --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/probmap_R_roi.shape.gii
Running: ciftify_seed_corr --weighted --hemi R --outputname /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_promap_giftitocifti_masked.dscalar.nii --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/probmap_R_roi.

0

In [127]:
run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_promap_giftitocifti_masked.dscalar.nii sub-50005 sub-50005_promap_giftitocifti_masked.dscalar.nii
Running: cifti_vis_map cifti-snaps --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_promap_giftitocifti_masked.dscalar.nii sub-50005 sub-50005_promap_giftitocifti_masked.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_promap_giftitocifti_masked.dscalar.nii
Running: ciftify_peaktable --min-threshold -0.5 --max-threshold 0.5 --no-cluster-dlabel /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_promap_giftitocifti_masked.dscalar.nii


In [128]:
result_map = seed_corr_default_out(func_cifti_smoothed, os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii'))
result_type = 'cifti'
result_prefix = '{}_tvertex7_ciftitocifti_unmasked.dscalar.nii'.format(subid)

run(['ciftify_seed_corr', '--roi-label', '7', func_cifti_smoothed, 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii')])

run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

In [131]:
result_map = os.path.join(seed_corr_dir, '{}_tvertex7_ciftitocifti_masked.dscalar.nii'.format(subid))
result_type = 'cifti'
result_prefix = '{}_tvertex7_ciftitocifti_masked.dscalar.nii'.format(subid)

run(['ciftify_seed_corr', '--roi-label', '7', 
     '--outputname', result_map,
     '--mask', cifti_mask, 
     func_cifti_smoothed, 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii')])

run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

In [134]:
R_tvertex_roi = os.path.join(new_outputs, 'rois', 'tvertex_R_roi.shape.gii')
result_map = seed_corr_default_out(func_cifti_smoothed, R_tvertex_roi)
result_type = 'cifti'
result_prefix = '{}_tvertex7_giftitocifti_umasked.dscalar.nii'.format(subid)


run(['wb_command', '-cifti-separate', 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii'),
     'COLUMN','-metric', 'CORTEX_RIGHT', R_tvertex_roi])
run(['ciftify_seed_corr', '--roi-label', '7', '--hemi', 'R', 
     func_cifti_smoothed, R_tvertex_roi])

run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

Running: wb_command -cifti-separate /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/tvertex.dscalar.nii COLUMN -metric CORTEX_RIGHT /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/tvertex_R_roi.shape.gii
Running: wb_command -cifti-separate /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/tvertex.dscalar.nii COLUMN -metric CORTEX_RIGHT /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/tvertex_R_roi.shape.gii
Running: ciftify_seed_corr --roi-label 7 --hemi R /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/tvertex_R_roi.shape.gii
Running: ciftify_seed_corr --roi-label 7 --hemi R /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s12.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/tvertex_R_roi.shape.gii


0

In [136]:
result_map = os.path.join(seed_corr_dir, '{}_tvertex7_giftitocifti_masked.dscalar.nii'.format(subid))
result_type = 'cifti'
result_prefix = '{}_tvertex7_giftitocifti_masked.dscalar.nii'.format(subid)

run(['ciftify_seed_corr', '--roi-label', '7', '--hemi', 'R', 
     '--outputname', os.path.join(seed_corr_dir, '{}_tvertex7_giftitocifti_masked.dscalar.nii'.format(subid)),
     '--mask', cifti_mask, 
     func_cifti_smoothed, R_tvertex_roi])

run_vis_map(result_map, result_prefix, result_type)
run_seedcorr_peaktable(result_map)

# ciftify_meants (atlas examples)

In [139]:
subject_aparc = os.path.join(hcp_data_dir, subid,
                            'MNINonLinear', 'fsaverage_LR32k',
                            '{}.aparc.32k_fs_LR.dlabel.nii'.format(subid))
subject_thickness = os.path.join(hcp_data_dir, subid,
                            'MNINonLinear', 'fsaverage_LR32k',
                            '{}.thickness.32k_fs_LR.dscalar.nii'.format(subid))
run(['ciftify_meants', func_cifti_sm0, subject_aparc])


Running: ciftify_meants /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.aparc.32k_fs_LR.dlabel.nii
Running: ciftify_meants /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.aparc.32k_fs_LR.dlabel.nii


0

In [140]:
run(['ciftify_meants',
     '--outputcsv', os.path.join(seed_corr_dir, '{}_aparc_func_ciftitocifti_masked_meants.csv'.format(subid)),
     '--mask', cifti_mask, 
     func_cifti_sm0, subject_aparc])

Running: ciftify_meants --outputcsv /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_aparc_func_ciftitocifti_masked_meants.csv --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.aparc.32k_fs_LR.dlabel.nii
Running: ciftify_meants --outputcsv /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_aparc_func_ciftitocifti_masked_meants.csv --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-5000

0

In [141]:
run(['ciftify_meants',  
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_aparc_thickness_ciftitocifti_unmasked_meants.csv'.format(subid)),
     '--outputlabels', os.path.join(seed_corr_dir, 
                                    '{}_aparc_ciftitocifti_unmasked_labels.csv'.format(subid)),
         subject_thickness, subject_aparc])

Running: ciftify_meants --outputcsv /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_aparc_thickness_ciftitocifti_unmasked_meants.csv --outputlabels /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_aparc_ciftitocifti_unmasked_labels.csv /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.thickness.32k_fs_LR.dscalar.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.aparc.32k_fs_LR.dlabel.nii
Running: ciftify_meants --outputcsv /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_aparc_thickness_ciftitocifti_unmasked_meants.csv --outputlabels /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_aparc_ciftitocifti_unmasked_labels.csv /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.thickness.32k_fs_LR.d

0

In [142]:
run(['ciftify_meants', 
     '--mask', cifti_mask,
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_aparc_thickness_ciftitocifti_unmasked_meants.csv'.format(subid)),
         subject_thickness, subject_aparc])

Running: ciftify_meants --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii --outputcsv /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_aparc_thickness_ciftitocifti_unmasked_meants.csv /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.thickness.32k_fs_LR.dscalar.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.aparc.32k_fs_LR.dlabel.nii
Running: ciftify_meants --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii --outputcsv /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_aparc_thickness_ciftitocifti_unmasked_meants.csv /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/fsaverage_LR32k/sub-50005.thickness.32k_fs_LR.dscalar.nii /home/edickie/Documents/ciftify_tests/r

0

In [143]:
run(['ciftify_meants', func_cifti_sm0, 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii')])

Running: ciftify_meants /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/tvertex.dscalar.nii
Running: ciftify_meants /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/tvertex.dscalar.nii


0

In [144]:
run(['ciftify_meants', 
     '--mask', cifti_mask,
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_tvertex_func_ciftitocifti_unmasked_meants.csv'.format(subid)),
     func_cifti_sm0, 
     os.path.join(new_outputs, 'rois', 'tvertex.dscalar.nii')])

Running: ciftify_meants --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii --outputcsv /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_tvertex_func_ciftitocifti_unmasked_meants.csv /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/tvertex.dscalar.nii
Running: ciftify_meants --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii --outputcsv /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_tvertex_func_ciftitocifti_unmasked_meants.csv /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/rois/tvertex.dscalar.nii


0

In [153]:
## project wmparc to subject
wmparc_dscalar_d0 = os.path.join(seed_corr_dir, '{}_wmparc_MNI_d0.dscalar.nii'.format(subid))
run(['ciftify_vol_result', '--hcp-data-dir', hcp_data_dir,
    '--integer-labels', subid, atlas_vol, wmparc_dscalar_d0])

Running: ciftify_vol_result --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --integer-labels sub-50005 /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/wmparc.nii.gz /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_wmparc_MNI_d0.dscalar.nii
Running: ciftify_vol_result --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --integer-labels sub-50005 /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/wmparc.nii.gz /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_wmparc_MNI_d0.dscalar.nii


0

In [156]:
## project wmparc to subject
wmparc_dscalar = os.path.join(seed_corr_dir, '{}_wmparc_MNI_d10.dscalar.nii'.format(subid))
run(['ciftify_vol_result', 
     '--hcp-data-dir', hcp_data_dir,
     '--dilate', '10',
    '--integer-labels', subid, atlas_vol, wmparc_dscalar])

Running: ciftify_vol_result --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --dilate 10 --integer-labels sub-50005 /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/wmparc.nii.gz /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_wmparc_MNI_d10.dscalar.nii
Running: ciftify_vol_result --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp --dilate 10 --integer-labels sub-50005 /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/wmparc.nii.gz /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_wmparc_MNI_d10.dscalar.nii


0

In [157]:
run(['ciftify_meants', 
     '--mask', os.path.join(hcp_data_dir, subid, 'MNINonLinear', 'brainmask_fs.nii.gz'),
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_niftitonifti_masked_meants.csv'.format(subid)),
     '--outputlabels', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_niftitonifti_masked_labels.csv'.format(subid)),
     func_cifti_sm0, atlas_vol])

Running: ciftify_meants --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/brainmask_fs.nii.gz --outputcsv /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_wmparc_func_niftitonifti_masked_meants.csv --outputlabels /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_wmparc_func_niftitonifti_masked_labels.csv /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/wmparc.nii.gz
Running: ciftify_meants --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/brainmask_fs.nii.gz --outputcsv /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_wmparc_func_niftitonifti_masked_meants.csv --outputlabels /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_wmpar

0

In [158]:
run(['ciftify_meants', 
     '--mask', cifti_mask,
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_ciftitocifti_masked_meants.csv'.format(subid)),
     '--outputlabels', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_ciftitocifti_masked_labels.csv'.format(subid)),
     func_cifti_sm0, wmparc_dscalar])

Running: ciftify_meants --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii --outputcsv /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_wmparc_func_ciftitocifti_masked_meants.csv --outputlabels /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_wmparc_func_ciftitocifti_masked_labels.csv /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1_Atlas_s0.dtseries.nii /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_wmparc_MNI_d10.dscalar.nii
Running: ciftify_meants --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii --outputcsv /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_wmparc_func_ciftitocifti_masked_meants.csv --outputlabels /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_

0

In [163]:
wmparc_dlabel = os.path.join(seed_corr_dir, '{}_wmparc_MNI.dlabel.nii'.format(subid))
run(['wb_command', '-cifti-label-import', '-logging', 'SEVERE',
     wmparc_dscalar, 
     os.path.join(ciftify.config.find_ciftify_global(), 'hcp_config', 'FreeSurferAllLut.txt'),
    wmparc_dlabel])
run(['ciftify_meants', 
     '--mask', cifti_mask,
     '--outputcsv', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_ciftiltocifti_masked_meants.csv'.format(subid)),
     '--outputlabels', os.path.join(seed_corr_dir, 
                                 '{}_wmparc_func_ciftiltocifti_masked_labels.csv'.format(subid)),
     func_cifti_sm0, wmparc_dlabel])

Running: wb_command -cifti-label-import -logging SEVERE /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_wmparc_MNI_d10.dscalar.nii /home/edickie/code/ciftify/ciftify/data/hcp_config/FreeSurferAllLut.txt /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_wmparc_MNI.dlabel.nii
Running: wb_command -cifti-label-import -logging SEVERE /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_wmparc_MNI_d10.dscalar.nii /home/edickie/code/ciftify/ciftify/data/hcp_config/FreeSurferAllLut.txt /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_wmparc_MNI.dlabel.nii
Running: ciftify_meants --mask /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_func_mask.dscalar.nii --outputcsv /home/edickie/Documents/ciftify_tests/run_2017-09-08/ciftify_seed_corr/sub-50005_wmparc_func_ciftiltocifti_masked_meants.csv --outputlabels /home/edickie/Documents/ciftify_test

0

In [160]:
run(['cifti_vis_map', 'index', '--hcp-data-dir', hcp_data_dir])

Running: cifti_vis_map index --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp
Running: cifti_vis_map index --hcp-data-dir /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp


0

In [168]:
### All results
# ciftify_peaktable - with va input
# cifti_vis_map
run(['extract_nuisance_regressors', 
     os.path.join(hcp_data_dir, subid, 'MNINonLinear'), 
     func_vol])


Running: extract_nuisance_regressors /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1.nii.gz
Running: extract_nuisance_regressors /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear /home/edickie/Documents/ciftify_tests/run_2017-09-08/hcp/sub-50005/MNINonLinear/Results/rest_test1/rest_test1.nii.gz


0

In [151]:
#group_nifti = '/home/edickie/Documents/ciftify_tests/src_data/working_memory_pFgA_z_FDR_0.01.nii.gz'
#group_cifti = '/home/edickie/Documents/ciftify_tests/run_2017-08-17/working_memory_pFgA_z_FDR_0.01.dscalar.nii'
#run(['ciftify_vol_result', '--debug', 'HCP_S1200_GroupAvg', group_nifti, group_cifti])

In [152]:
recon_all_always = sorted(set(test3).intersection(set(test1)))

NameError: name 'test3' is not defined

In [ ]:
onlytest2 = set(test2) - set(test1)
recon_all_lowres = [ x for x in onlytest2 if 'T1w' in x ]
sorted(recon_all_lowres)

In [ ]:
only_in_5 = set(test3) - set(test2)

In [ ]:
tmp1 = set(test2) - set(test3)
tmp1 = [ x for x in tmp1 if 'T1w' not in x ]
tmp1 = [ x for x in tmp1 if 'Results' not in x ]
tmp1 = [ x for x in tmp1 if 'aseg' not in x ]
only_in_6 = sorted(tmp1)
only_in_6

In [ ]:
ciftify_recon_all = { 'always': recon_all_always,
                    'T1w32k': recon_all_lowres,
                    'fs5' : only_in_5,
                    'fs6' : only_in_6}

In [ ]:
with open('/home/edickie/code/ciftify/tests/integration/data/ciftify_recon_all.yml', 'w') as outfile:
    yaml.dump(ciftify_recon_all, outfile, default_flow_style=False)

In [ ]:
with open('data.yml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)